In [1]:
#importing packages
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('ggplot')
from scipy.stats import norm
from scipy import stats
%matplotlib inline

In [2]:
#so all DataFrame data is displayed
pd.set_option('display.max_columns',1000)
pd.set_option('display.min_rows',0)
pd.set_option('display.max_rows',100)

In [3]:
ZRI = pd.read_csv('./data/Zip_Zri_MultiFamilyResidenceRental.csv', index_col=0,dtype={'RegionName': 'str'})


In [60]:
ACS = pd.read_pickle('./data/ZRI_ACS.pkl')

In [65]:
ACS['ZIP_CODE'] = ACS['ZIP_CODE'].astype(str)

In [66]:
#make all zip codes be 5 digits
ACS['ZIP_CODE'] = ACS['ZIP_CODE'].str.rjust(5, '0')


In [95]:
ZRI_IRS_LF_MD = pd.read_pickle('./data/ZRI_IRS_LF_MD.pkl')

In [117]:
#season trends 

def month_to_season(month):
    if month in range(3,5):
        month = 'spring'
    elif month in range(6,8):
        month = 'summer'
    elif month in range(9,11):
        month = 'fall'
    else:
        month = 'winter'
    return(month)

ZRI_IRS_LF_MD['Season'] = ZRI_IRS_LF_MD.Month.apply(month_to_season)

In [119]:
ZRI_IRS_LF_MD_17 = ZRI_IRS_LF_MD[ZRI_IRS_LF_MD.Year == 2017]

In [120]:
ACS.columns

Index(['ZIPCODE', 'City', 'State', 'Metro', 'CountyName', 'SizeRank',
       'Year-Month', 'value', 'Year', 'ZCTA_x', 'RentalVacancyRate', 'white',
       'black', 'asian', 'NHOPI', 'hispanic_latino', '20_to_24', '25_to_34',
       '35_to_44', '45_to_54', '55_to_59', '60_to_64', '65_to_74', '75_to_85',
       '85_over', 'Population'],
      dtype='object')

In [121]:
ACS.rename(columns={"ZIP_CODE": "ZIPCODE", 'date':'Year-Month', 'year':'Year'}, inplace=True)
#datetime
ACS['Year-Month'] = pd.to_datetime(ACS['Year-Month'])

ACS_17 = ACS[ACS.Year ==2017]

In [122]:
ZRI_IRS_LF_MD_17.columns

Index(['ZIPCODE', 'City', 'State', 'Metro', 'CountyName', 'SizeRank',
       'Year-Month', 'value', 'Month', 'Year', 'Season', 'AGIncome',
       'TotIncome', 'SalariesWages', 'NumUnemply', 'TotTaxes', 'FIPSCode',
       'Labor\nForce', 'Employed', 'Unemployed', 'Unemploy-\nment Rate\n(%)',
       'MortDeliq30-89', 'MortDeliq90'],
      dtype='object')

In [123]:
ACS_17.columns


Index(['ZIPCODE', 'City', 'State', 'Metro', 'CountyName', 'SizeRank',
       'Year-Month', 'value', 'Year', 'ZCTA_x', 'RentalVacancyRate', 'white',
       'black', 'asian', 'NHOPI', 'hispanic_latino', '20_to_24', '25_to_34',
       '35_to_44', '45_to_54', '55_to_59', '60_to_64', '65_to_74', '75_to_85',
       '85_over', 'Population'],
      dtype='object')

In [124]:
ZRI_IRS_LF_MD_17.head(1)

,ZIPCODE,City,State,Metro,CountyName,SizeRank,Year-Month,value,Month,Year,Season,AGIncome,TotIncome,SalariesWages,NumUnemply,TotTaxes,FIPSCode,Labor\nForce,Employed,Unemployed,Unemploy-\nment Rate\n(%),MortDeliq30-89,MortDeliq90
19155528,10025,New York,NY,New York-Newark-Jersey City,New York County,1,2017-01-01,3771.0,1,2017,winter,1.222721e+06,1247542.5,763333.333333,208.333333,138435.833333,36061,"916,625","878,936","37,689",4.1,0.8,0.7


In [125]:
ACS_17.head(1)

,ZIPCODE,City,State,Metro,CountyName,SizeRank,Year-Month,value,Year,ZCTA_x,RentalVacancyRate,white,black,asian,NHOPI,hispanic_latino,20_to_24,25_to_34,35_to_44,45_to_54,55_to_59,60_to_64,65_to_74,75_to_85,85_over,Population
141436,10025,New York,NY,New York-Newark-Jersey City,New York County,1,2017-01-01,3771.0,2017,10025,1.1,64.4,12.5,9.8,0.0,94382,6.6,18.0,13.2,13.1,5.9,7.3,11.5,4.8,3.0,94382


In [182]:
#merge ZRI_IRS_LF_MD_ACS 2017 to make X train
X_train17 = pd.merge(ZRI_IRS_LF_MD_17, ACS_17, on=['ZIPCODE', 'City', 'State', 'Metro', 'CountyName', 'SizeRank', 'Year-Month', 'value', 'Year'], how='left')


In [183]:
X_train17.shape

(3192588, 40)

In [196]:
#dropping duplicates 
X_train17 = X_train17.drop_duplicates()
X_train17.shape #checking shape

(66888, 29)

In [127]:
ZRI_IRS_LF_MD_18 = ZRI_IRS_LF_MD[ZRI_IRS_LF_MD.Year == 2018]

In [128]:
ACS_18 = ACS[ACS.Year ==2018]

In [184]:
#merge ZRI_IRS_LF_MD_ACS 2018 Y train
y_train18 = pd.merge(ZRI_IRS_LF_MD_18, ACS_18, on=['ZIPCODE', 'City', 'State', 'Metro', 'CountyName', 'SizeRank', 'Year-Month', 'value', 'Year'], how='left')


In [185]:
y_train18.shape

(3192588, 40)

In [187]:
#dropping duplicates 
y_train18 = y_train18.drop_duplicates()
y_train18.shape #checking shape

(66888, 40)

# Cleaning columns 
- removing unnecessary columns

In [170]:
X_train17.columns

Index(['ZIPCODE', 'City', 'State', 'Metro', 'CountyName', 'SizeRank',
       'Year-Month', 'value', 'Month', 'Year', 'Season', 'AGIncome',
       'TotIncome', 'SalariesWages', 'NumUnemply', 'TotTaxes', 'FIPSCode',
       'Labor\nForce', 'Employed', 'Unemployed', 'Unemploy-\nment Rate\n(%)',
       'MortDeliq30-89', 'MortDeliq90', 'ZCTA_x', 'RentalVacancyRate', 'white',
       'black', 'asian', 'NHOPI', 'hispanic_latino', '20_to_24', '25_to_34',
       '35_to_44', '45_to_54', '55_to_59', '60_to_64', '65_to_74', '75_to_85',
       '85_over', 'Population'],
      dtype='object')

In [188]:
season_2017 = X_train17.groupby(['Season']).agg({"value": 'median'})
season_all = ZRI_IRS_LF_MD.groupby(['Season']).agg({"value": 'median'})

In [189]:
season_all

,value
Season,
fall,1572.0
spring,1525.0
summer,1545.0
winter,1552.0


In [190]:
#making season ordinal 
X_train17.Season = X_train17.Season.replace({'fall':1,'winter':2, 'summer':3, 'spring':4})

In [ ]:
#X_train17.head()

In [209]:
#drop columns to test
X_train17.drop(columns = ['City', 'State', 'Metro', 'CountyName','Month', 'Year', 'Season','ZCTA_x'], inplace=True)
X_train17.drop(columns = ['Labor\nForce','Employed', 'Unemployed'], inplace=True)

X_train17.drop(columns = ['Year-Month'], inplace=True)


X_train17.columns

Index(['ZIPCODE', 'SizeRank', 'value', 'AGIncome', 'TotIncome',
       'SalariesWages', 'NumUnemply', 'TotTaxes', 'FIPSCode',
       'Unemploy-\nment Rate\n(%)', 'MortDeliq30-89', 'MortDeliq90',
       'RentalVacancyRate', 'white', 'black', 'asian', 'NHOPI',
       'hispanic_latino', '20_to_24', '25_to_34', '35_to_44', '45_to_54',
       '55_to_59', '60_to_64', '65_to_74', '75_to_85', '85_over',
       'Population'],
      dtype='object')

In [211]:
X_train = X_train17.copy()

In [151]:
y_train18 = y_train18.dropna(subset=['value'])

In [201]:
y_train = y_train18['value']

In [197]:
X_train17.shape

(66888, 29)

In [212]:
X_train.shape

(66888, 28)

In [213]:
y_train.shape

(66888,)

In [214]:
#replace nan with 0 
X_train = X_train.replace(np.nan, 0)
y_train = y_train.replace(np.nan, 0)

In [215]:
X_train.shape

(66888, 28)

In [216]:
y_train.shape

(66888,)

In [217]:
X_train.head()

,ZIPCODE,SizeRank,value,AGIncome,TotIncome,SalariesWages,NumUnemply,TotTaxes,FIPSCode,Unemploy-\nment Rate\n(%),MortDeliq30-89,MortDeliq90,RentalVacancyRate,white,black,asian,NHOPI,hispanic_latino,20_to_24,25_to_34,35_to_44,45_to_54,55_to_59,60_to_64,65_to_74,75_to_85,85_over,Population
0,10025,1,3771.0,1.222721e+06,1.247542e+06,7.633333e+05,208.333333,138435.833333,36061,4.1,0.8,0.7,1.1,64.4,12.5,9.8,0.0,94382,6.6,18.0,13.2,13.1,5.9,7.3,11.5,4.8,3.0,94382
129,60657,2,1770.0,8.573410e+05,8.710963e+05,6.170520e+05,161.666667,46717.166667,13075,4.5,0.0,0.0,1.3,86.6,2.9,6.7,0.0,70103,11.2,38.0,14.1,9.5,3.7,3.3,4.9,2.4,1.0,70103
134,60657,2,1770.0,8.573410e+05,8.710963e+05,6.170520e+05,161.666667,46717.166667,17031,5.1,2.2,1.9,1.3,86.6,2.9,6.7,0.0,70103,11.2,38.0,14.1,9.5,3.7,3.3,4.9,2.4,1.0,70103
360,60657,2,1770.0,8.573410e+05,8.710963e+05,6.170520e+05,161.666667,46717.166667,27031,3.8,0.0,0.0,1.3,86.6,2.9,6.7,0.0,70103,11.2,38.0,14.1,9.5,3.7,3.3,4.9,2.4,1.0,70103
366,10023,3,3490.0,2.270603e+06,2.299776e+06,1.031574e+06,145.000000,303624.166667,36061,4.1,0.8,0.7,1.8,79.8,3.7,11.0,0.0,61514,3.9,17.8,16.6,13.3,6.6,5.8,12.0,5.4,3.5,61514


# Gradient Boost

In [218]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [220]:
gbm = GradientBoostingRegressor()

gbm.set_params(n_estimators = 464, 
            learning_rate = .1, #.3,
            max_depth = 2, #2,
            min_samples_leaf = 3)

gbm.fit(X_train,y_train)

print('-'*50)
print("The train set R^2 is %.3f" %(gbm.score(X_train, y_train)))
#print("The test set R^2 is %.3f" %(gbm.score(X_test,y_test)))

--------------------------------------------------
The train set R^2 is 0.857


### Grid Search for Gradient Boost

In [221]:
# n_estimator: 

x = np.logspace(1, 4, 100)
x = x.astype(np.int64)

In [ ]:
param_grid={'n_estimators':x,
           'learning_rate': [.3,.2,0.1],
            'max_depth':[1,2,3],
            'min_samples_leaf':[3]}

grid_search = GridSearchCV(estimator= gbm, param_grid= param_grid)
grid_search.fit(X_train,y_train)

In [ ]:
print(grid_search.best_estimator_)
print(grid_search.best_params_)
print(grid_search.best_score_)

In [ ]:
gbm.get_params